In [100]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

In [22]:
# driver = webdriver.Chrome()
chrome_options = Options()
driver = uc.Chrome(options=chrome_options)

### 知乎搜索 cpr 女性

In [72]:
# 滑轮下拉到底部
def scroll_down(driver, XPATH_INFO):
    items = driver.find_elements(By.XPATH, XPATH_INFO)
    prev_len = 0
    curr_len = len(items)
    first_time = 0

    while True:
        if prev_len != curr_len or first_time < 2:
            # 上滑下滑防止加载不出来
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            items = driver.find_elements(By.XPATH, XPATH_INFO)
            prev_len = curr_len
            curr_len = len(items)
            first_time += 1
        else:
            break 
    
    return items

In [ ]:
driver.get('https://www.zhihu.com/search?type=content&q=cpr%20%E5%A5%B3%E6%80%A7')
time.sleep(3)

In [75]:
# 下拉到底部, 加载所有数据
items = scroll_down(driver, '//div[@class="Card SearchResult-Card"]')

In [ ]:
df = pd.DataFrame(columns=['title', 'content', 'comment number', 'link'])

In [ ]:
for item in items:
    # 如果有阅读全文, 点击阅读全文
    try:
        read_more = item.find_element(By.XPATH, './/button[@class="Button ContentItem-more FEfUrdfMIKpQDJDqkjte Button--plain fEPKGkUK5jyc4fUuT0QP"]')
        read_more.click()
        time.sleep(2)
    except:
        0

    # 获取标题和链接
    try:
        title = item.find_element(By.XPATH, './/span[@class="Highlight"]').text
        link = item.find_element(By.XPATH, './/a[@target="_blank"]').get_attribute('href')
        content = item.find_element(By.XPATH, './/div[@class="RichContent-inner"]').text
        df.loc[len(df), 'title'] = title
        df.loc[len(df) - 1, 'link'] = link
        df.loc[len(df) - 1, 'content'] = content
    except:
        0

    # 获取评论数
    try:
        comment_number = item.find_element(By.XPATH, './/button[@class="Button ContentItem-action FEfUrdfMIKpQDJDqkjte Button--plain Button--withIcon Button--withLabel fEPKGkUK5jyc4fUuT0QP B46v1Ak6Gj5sL2JTS4PY RuuQ6TOh2cRzJr6WlyQp"]').text
        df.loc[len(df) - 1, 'comment number'] = comment_number
    except:
        0

In [ ]:
df.to_csv('zhihu.csv', index=False, encoding='utf-8-sig')
df.head(2)

### 清理数据

In [3]:
def replace_with_nan(text):
    if isinstance(text, str) and '添加评论' in text:
        return np.nan
    return text

In [4]:
def split_and_convert_to_int(text):
    if isinstance(text, str):
        split_text = text.split('条')[0]
        try:
            return int(split_text)
        except ValueError:
            0
    return text

In [5]:
zh_df = pd.read_csv('zhihu.csv')
# comment number替换添加评论为空值
zh_df['comment number'] = zh_df['comment number'].apply(replace_with_nan)
# comment number按照条字分割, 取第一个值，并且不是空值把它转成int类型
zh_df['comment number'] = zh_df['comment number'].apply(split_and_convert_to_int)
# 按照评论数排序并且重新设定索引
zh_df = zh_df.sort_values(by='comment number', ascending=False).reset_index(drop=True)
zh_df.head(5)

,title,content,comment number,link
0,怎么给戴的bra的女性做CPR？,鸡糟的黄医桑：为毛要邀请我回答这种问题？ 我是正经医生，不开车的(๑˙ー˙๑) 自己看图啊—...,75.0,https://www.zhihu.com/question/56807265/answer...
1,假如你是一个本科刚毕业的医学生，路上突然有人需要CPR，你会怎么办？,讲一个真实的案例。\n一个医生，出去游泳，遇到人溺水，CPR之后把溺水的人拖回医院抢救，没抢...,11.0,https://www.zhihu.com/question/433206267/answe...
2,如何看待火车站女大学生行CPR跪地救人的微博评价？,不邀自来。\n利益相关：曾经的4年临床急诊ICU护士。\n首先，对楼主被责骂表示同情。没错，...,8.0,https://www.zhihu.com/question/286371378/answe...
3,女朋友是护士经常会给别的异性做人工呼吸，有点接受不了怎么办？,malus：在医院的话多半直接拐急救进icu了，哪轮得到您女朋友做人工呼吸呢？ 在室外？那没...,6.0,https://www.zhihu.com/question/478361963/answe...
4,要不要给路上遇到的昏倒女性做CPR，即心肺复苏？,我们会有猫的：终于遇到与本专业有关可以回答的题了！！！ 在中国受过专业心肺复苏训练的人不到百...,3.0,https://www.zhihu.com/question/29457803/answer...


### 选择“怎么给戴的bra的女性做CPR？”和“要不要给路上遇到的昏倒女性做CPR，即心肺复苏？”两个标题来抓去评论

In [89]:
# 获取第一条和第五条数据的链接，并保存到list
links = zh_df.loc[[0, 4], 'link'].tolist()

link = links[0]
driver.get(link)
time.sleep(2)

In [90]:
# 如果有展开全部评论，点击展开全部评论
try:
    expand_all = driver.find_element(By.XPATH, './/a[@class="QuestionMainAction ViewAll-QuestionMainAction"]')
    expand_all.click()
    time.sleep(2)
except:
    0

# 滑轮下拉到底部
items = scroll_down(driver, './/div[@class="List-item"]') 

In [101]:
# 初始化
df = pd.DataFrame(columns=['topic', 'comment', 'agree number'])

In [102]:
# 获取话题 + 评论 + 赞同数
for item in items:
    # 获取话题
    try:
        topic = item.find_element(By.XPATH, './/div[@class="RichContent-inner"]').text
        df.loc[len(df), 'topic'] = topic
    except:
        0

    # 获取赞同数
    try:
        agree_number = item.find_element(By.XPATH, './/span[@class="Voters"]//button[@type="button"]').text.split(' ')[0]
        df.loc[len(df) - 1, 'agree number'] = agree_number
    except:
        0

    # 如果有评论，展开评论
    try:     
        expand_comment = item.find_elements(By.XPATH, './/div[@class="ContentItem-actions"]//button[@type="button"]')[2]
        if '添加评论' not in expand_comment.text and '收起评论' not in expand_comment.text:
            expand_comment.click()
            time.sleep(2)
    except:
        0

    # 获取评论
    try:
        comment = ''
        for sub_item in item.find_elements(By.XPATH, './/div[@class="css-14nvvry"]//div[@class="CommentContent css-1ygdre8"]'):
            comment += sub_item.text
        df.loc[len(df) - 1, 'comment'] = comment
    except:
        0

In [108]:
df.to_csv('zhihu_topic_1.csv', index=False, encoding='utf-8-sig')
df.head(3)

,topic,comment,agree number
0,为毛要邀请我回答这种问题？\n我是正经医生，不开车的(๑˙ー˙๑)\n自己看图啊——\n\n...,薛定谔的巨乳妹子:处于妹子与尸体的中间状态，如果不动色心抢救可能还是巨乳妹子，如果动色心延误...,283
1,设定：\n在地铁站发现一美女突然昏倒\n先呼救，确定人没有呼吸和脉搏了和意识，打120\n把...,,55
2,要把bra脱掉，为了后续用AED方便，因为AED需要裸露的皮肤且避免金属导电。\n但是要注意...,急救课上有一项就是人文关怀(˶‾᷄ ⁻̫ ‾᷅˵)做完CPR后一定要把人家衣服盖好，注意保护...,14


In [109]:
link = links[1]
driver.get(link)
time.sleep(2)

In [110]:
# 如果有展开全部评论，点击展开全部评论
try:
    expand_all = driver.find_element(By.XPATH, './/a[@class="QuestionMainAction ViewAll-QuestionMainAction"]')
    expand_all.click()
    time.sleep(2)
except:
    0

# 滑轮下拉到底部
items = scroll_down(driver, './/div[@class="List-item"]') 

In [111]:
# 初始化
df = pd.DataFrame(columns=['topic', 'comment', 'agree number'])

In [112]:
# 获取话题 + 评论 + 赞同数
for item in items:
    # 获取话题
    try:
        topic = item.find_element(By.XPATH, './/div[@class="RichContent-inner"]').text
        df.loc[len(df), 'topic'] = topic
    except:
        0

    # 获取赞同数
    try:
        agree_number = item.find_element(By.XPATH, './/span[@class="Voters"]//button[@type="button"]').text.split(' ')[0]
        df.loc[len(df) - 1, 'agree number'] = agree_number
    except:
        0

    # 如果有评论，展开评论
    try:     
        expand_comment = item.find_elements(By.XPATH, './/div[@class="ContentItem-actions"]//button[@type="button"]')[2]
        if '添加评论' not in expand_comment.text and '收起评论' not in expand_comment.text:
            expand_comment.click()
            time.sleep(2)
    except:
        0

    # 获取评论
    try:
        comment = ''
        for sub_item in item.find_elements(By.XPATH, './/div[@class="css-14nvvry"]//div[@class="CommentContent css-1ygdre8"]'):
            comment += sub_item.text
        df.loc[len(df) - 1, 'comment'] = comment
    except:
        0

In [113]:
df.to_csv('zhihu_topic_2.csv', index=False, encoding='utf-8-sig')
df.head(3)

,topic,comment,agree number
0,其实衣服未必需要全部扒开（最关键的在衣领，目的在于开放气道），而且严格来说口对口人工呼吸也是...,口对口还能传染艾滋?我书读的少，你不要骗我有过因为接吻感染AIDS的事例，推测原因是嘴唇或口...,10
1,终于遇到与本专业有关可以回答的题了！！！\n在中国受过专业心肺复苏训练的人不到百分之一，所以...,嗯？！为什么答主说按压的部位是在两乳连线下方啊……表示这几天看的教学视频告诉我是在两乳连线的...,13
2,1.从社会角度讲\n一定要救！\n因为如果碰到有需要急救的情况，知道急救手段，有急救条件而不...,,2


In [ ]:
driver.close()